In [4]:
import pandas as pd
import numpy as np
import requests
import json
import time
import os
from datetime import datetime
import random


key = "3xP6AGMGXlUwMGUkKfIOCDDwrBOLT7Z0"
url = "https://elevation-api.io/api/elevation?points="
json_file_path = "data/data_lookup_params.json"
loc_url = "https://nominatim.openstreetmap.org/reverse?format=jsonv2&key={}".format(key)


class GenerateDataset():
    def __init__(self, file_path, test_effect):
        self.fp = file_path
        self.test_effect = test_effect
        self.Dataset = self.generate_dataset(sample=True)
        
    def generate_dataset(self, sample):
        """
        generates mock sensor_data to be tested
        in the scope model, data sufficiency & compliancy
        :param file_path:
        :return: dataframe
        """

        print("GENERATING ANNOTATED FOR THE POINTS")
        df = pd.read_csv(self.fp + self.test_effect)
        print(df.head())
        #df.reindex(columns=[*df.columns.tolist(), 'Latitude', 'Longitude', 'Datetime', 'SPEED'], fill_value=0)
        
        if sample:
            df = df.sample(n=1000, random_state=1)
        
        print("GENERATING {} sample".format(len(df)))
    
        with open(json_file_path) as json_data:
            # @note -> df[shape] can be changed to whatever size we want to randomize
            d = json.load(json_data)
            df['Latitude'] = np.random.uniform(d["latitude"][0], d["latitude"][1], df.shape[0])
            df['Longitude'] = np.random.uniform(d['longitude'][0], d['longitude'][1], df.shape[0])
            df['Coordinates'] = tuple(zip(df['Latitude'], df['Longitude']))
            df['Datetime'] = np.random.randint(d['datetime'][0], d['datetime'][1], df.shape[0])
            df['formattedTime'] = pd.to_datetime(df['Datetime'], unit='s')
            df['Speed'] = np.random.uniform(0,130, df.shape[0])            
            df['month'] = pd.to_datetime(df['formattedTime']).dt.strftime('%m')
            environ_dict = d['environmental']
            temperature_lists = self.generate_environmental_data(df, environ_dict)
            df['Temperature'] = temperature_lists[0]
            df['Precipitation'] = temperature_lists[1]
            
            
        print("ELEVATION DATA GENERATING")
        ## not now
        df['Elevation'] = self.calculate_elevations(df)
        #df['Open_Data'] = ""
        
        ## to do -> fix the timeout here

        print("GENERATING GEOGRAPHICAL PROFILE & RoadType")
        df['Open_Data'] = self.calculate_open_data(df)

        df['RoadTypes'] = self.get_road_types(df)
            
        # ImageBased Elements
        df.to_csv(self.fp + "scope_annotated_" + self.test_effect , index=False)
        return df


    def calculate_open_data(self, df):
        """
        gathers the open_Data points on the location, including the region, & name
        :param df: coordinates
        :return: list of open-data json objects
        """
        print("COLLECTING OPEN DATA FROM ENDPOINT")

        open_map_data = list()
        count = 0
        for i in df.index:
            val = df.loc[i,'Coordinates']
            revised_url = \
                loc_url + "&lat=" + str(val[0]) + "&lon=" + str(val[1])
            try:
                r = requests.get(revised_url, headers={'User-agent': '0.' + str(count)})
            except requests.exceptions.HTTPError as err:
                r = err
            print(r.text)
            open_map_data.append(r.text)
            count = count + 1
            if count % 20 == 0:
                time.sleep(20)
        return open_map_data


    def calculate_elevations(self, df):
        """
        uses the elevation API to calculate the elevations based on the
        coordinates

        :param df:
        :return: [] list of altitudes
        """

        elevations = list()
        for k,g in df.groupby(np.arange(len(df))//10):
            response_body = ""
            for element in g['Coordinates']:
                response_body = response_body + str(element).replace(" ", "") + ","
            response = json.loads(requests.get(url + response_body).text)['elevations']
            for index in range(len(response)):
                elevations.append(response[index]['elevation'])

        return elevations


    def generate_environmental_data(self, df, monthly_enviro_dict):
        temperature_generator = list()
        precipitation_generator = list()
        count = 0
        for month in df['month']:
            for event in monthly_enviro_dict['temp']:
                if month == event:
                    temperature_generator.append(
                        np.random.uniform(monthly_enviro_dict['temp'][event][0], monthly_enviro_dict['temp'][event][1]))
                    precipitation_generator.append(
                        np.random.uniform(monthly_enviro_dict['precip'][event][0], monthly_enviro_dict['precip'][event][1]))
                # adds random noise
                if count % 550:
                    temperature_generator.append(40)
                    precipitation_generator.append(120)
        return temperature_generator, precipitation_generator


    def get_road_types(self, df):
        road_types = list()
        for index, row in df.iterrows():
            road_type = json.loads(row['Open_Data']).get('addresstype')
            if not road_type:
                road_type = "unknown"
            road_types.append(road_type)
        return road_types

    
base_path = "/Users/luciaeve/Documents/EMSE/KAISERSLAUTERN/THESIS/code/GTSRB_augmentations/Augmented/output_data/RANDOM_IMAGES/COMPILED_TESTSET/VALIDATION/"
test_effect = "ALL_combined.csv"
g = GenerateDataset(base_path, test_effect)

GENERATING ANNOTATED FOR THE POINTS
   AIR_PRESSURE  APPROACHING_TRAFFIC  BACKLIGHTSUN  CLOUDINESS  DARKNESS  \
0         960.0                  0.0          25.0        -1.0       0.0   
1         790.0                  0.0          25.0        -1.0       0.0   
2         644.0                  0.0          25.0        -1.0       0.0   
3        1036.0                  0.0          25.0         5.0       0.0   
4         812.0                  0.0          25.0         6.0       0.0   

                  DATE  DIRTLENS  GRADIENT  HEADLIGHTS_OWN  HUMIDITY  ...  \
0  2019-12-05 00:00:00      25.0       0.0             0.0      23.0  ...   
1  2019-12-05 00:00:00      25.0       0.0             0.0      59.0  ...   
2  2019-12-05 00:00:00      25.0       0.0             0.0      90.0  ...   
3  2019-12-05 00:00:00      25.0       0.0             0.0       5.0  ...   
4  2019-12-05 00:00:00      25.0       0.0             0.0       6.0  ...   

       Unnamed: 0  VEHICLESPEED  VISIBILITY 

{"place_id":5960172,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"node","osm_id":598221342,"lat":"53.6093508","lon":"11.3818508","place_rank":30,"category":"highway","type":"bus_stop","importance":0,"addresstype":"highway","name":"Görries Brücke","display_name":"Görries Brücke, B 106, Dwang, Görries, Neu Pampow, Schwerin, Mecklenburg-Vorpommern, 19061, Deutschland","address":{"bus_stop":"Görries Brücke","road":"B 106","neighbourhood":"Dwang","suburb":"Görries","village":"Neu Pampow","city":"Schwerin","state":"Mecklenburg-Vorpommern","postcode":"19061","country":"Deutschland","country_code":"de"},"boundingbox":["53.6092508","53.6094508","11.3817508","11.3819508"]}
{"place_id":87046123,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":42467831,"lat":"48.615974347726","lon":"10.9555114948824","place_rank":26,"category":"highway","type":"residential","importance":0.1,"addresstype":"r

{"place_id":249038139,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":659179316,"lat":"51.56639805","lon":"12.9230698194547","place_rank":30,"category":"building","type":"yes","importance":0,"addresstype":"building","name":null,"display_name":"20, Mühlenweg, Bubendorf, Dreiheide, Torgau, Nordsachsen, Sachsen, 04860, Deutschland","address":{"house_number":"20","road":"Mühlenweg","suburb":"Bubendorf","city":"Dreiheide","county":"Torgau","state":"Sachsen","postcode":"04860","country":"Deutschland","country_code":"de"},"boundingbox":["51.5663417","51.5664542","12.9229786","12.923161"]}
{"place_id":95841969,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":90972137,"lat":"52.7949419980068","lon":"12.8042253426019","place_rank":26,"category":"highway","type":"residential","importance":0.1,"addresstype":"road","name":"Heckenweg","display_name":"Heckenweg, Tarmow, Fehrbellin,

{"place_id":84219995,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":32643192,"lat":"49.2931927312924","lon":"6.10916205104034","place_rank":26,"category":"highway","type":"residential","importance":0.1,"addresstype":"road","name":"Rue de la Forêt","display_name":"Rue de la Forêt, Fameck, Thionville, Moselle, Grand Est, France métropolitaine, 57290, France","address":{"road":"Rue de la Forêt","town":"Fameck","county":"Thionville","state":"Grand Est","country":"France","postcode":"57290","country_code":"fr"},"boundingbox":["49.2918896","49.2945143","6.1048804","6.1127846"]}
{"place_id":198097880,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"relation","osm_id":47796,"lat":"52.24726925","lon":"5.54122345487672","place_rank":6,"category":"boundary","type":"administrative","importance":0.85279929223856,"addresstype":"boundary","name":"Nederland","display_name":"Nederland","address":

{"place_id":198687728,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"relation","osm_id":3650673,"lat":"52.6547123","lon":"11.8956412","place_rank":18,"category":"boundary","type":"administrative","importance":0.242383665382304,"addresstype":"boundary","name":"Jarchau","display_name":"Jarchau, Stendal, Sachsen-Anhalt, Deutschland","address":{"city_district":"Jarchau","town":"Stendal","county":"Stendal","state":"Sachsen-Anhalt","country":"Deutschland","country_code":"de"},"boundingbox":["52.6421364","52.677298","11.8622495","11.9098843"]}
{"place_id":4747975,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"node","osm_id":505315186,"lat":"48.6153435","lon":"10.8706052","place_rank":19,"category":"place","type":"hamlet","importance":0.275,"addresstype":"place","name":"Gut Herrlehof","display_name":"Gut Herrlehof, Nordendorf, Landkreis Augsburg, Schwaben, Bayern, 86679, Deutschland","address":{"ham

...

{"place_id":88185821,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":47124295,"lat":"51.9348500617401","lon":"7.8633513972369","place_rank":26,"category":"highway","type":"tertiary","importance":0.1,"addresstype":"road","name":"Müssingen","display_name":"Müssingen, Schmaler Kamp, Everswinkel, Kreis Warendorf, Regierungsbezirk Münster, Nordrhein-Westfalen, 48351, Deutschland","address":{"road":"Müssingen","neighbourhood":"Schmaler Kamp","village":"Everswinkel","county":"Kreis Warendorf","state_district":"Regierungsbezirk Münster","state":"Nordrhein-Westfalen","postcode":"48351","country":"Deutschland","country_code":"de"},"boundingbox":["51.9347499","51.9458995","7.8589375","7.8974101"]}
{"place_id":81809510,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":28186892,"lat":"51.5077212648268","lon":"8.02457508164207","place_rank":27,"category":"highway","type":"service",

{"place_id":124242059,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":208251558,"lat":"50.1976447300395","lon":"6.98380703071379","place_rank":26,"category":"highway","type":"primary","importance":0.1,"addresstype":"road","name":"B 259","display_name":"B 259, Waldfrieden, Auderath, Ulmen, Landkreis Cochem-Zell, Rheinland-Pfalz, 56766, Deutschland","address":{"road":"B 259","suburb":"Waldfrieden","village":"Auderath","county":"Ulmen","state":"Rheinland-Pfalz","postcode":"56766","country":"Deutschland","country_code":"de"},"boundingbox":["50.193064","50.2014632","6.9798839","7.0060612"]}
{"place_id":88121364,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":46898789,"lat":"50.9060804","lon":"8.7459503","place_rank":26,"category":"highway","type":"unclassified","importance":0.1,"addresstype":"road","name":"Hügelsbirke","display_name":"Hügelsbirke, Aumühle, Wetter, Landkr

{"place_id":169117962,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":426400900,"lat":"49.264017","lon":"6.893154","place_rank":27,"category":"highway","type":"service","importance":0.075,"addresstype":"road","name":"Auf Rotstay","display_name":"Auf Rotstay, Ritterstraße, Püttlingen, Regionalverband Saarbrücken, Saarland, 66346, Deutschland","address":{"road":"Auf Rotstay","suburb":"Ritterstraße","city_district":"Püttlingen","town":"Püttlingen","county":"Regionalverband Saarbrücken","state":"Saarland","postcode":"66346","country":"Deutschland","country_code":"de"},"boundingbox":["49.2638173","49.2661231","6.8931422","6.8937991"]}
{"place_id":160242111,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":379373832,"lat":"50.3431763","lon":"11.3181305060349","place_rank":30,"category":"building","type":"yes","importance":0,"addresstype":"building","name":null,"display_name

{"place_id":183183473,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":513368344,"lat":"49.6798541193542","lon":"6.53993338400126","place_rank":26,"category":"highway","type":"primary","importance":0.1,"addresstype":"road","name":"B 51","display_name":"B 51, Könen, Konz, Landkreis Trier-Saarburg, Rheinland-Pfalz, 54329, Deutschland","address":{"road":"B 51","suburb":"Könen","town":"Konz","county":"Konz","state":"Rheinland-Pfalz","postcode":"54329","country":"Deutschland","country_code":"de"},"boundingbox":["49.6767183","49.683204","6.5392148","6.5406392"]}
{"place_id":86233092,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":38753651,"lat":"53.7439534374604","lon":"10.0242315912784","place_rank":26,"category":"highway","type":"unclassified","importance":0.1,"addresstype":"road","name":"Kringelweg","display_name":"Kringelweg, Tangstedt, Itzstedt, Stormarn, Schleswig-Ho

{"place_id":95317876,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":89811713,"lat":"53.4983499","lon":"9.11752906319096","place_rank":30,"category":"building","type":"yes","importance":0,"addresstype":"building","name":"GETI Wilba","display_name":"GETI Wilba, 2, Hansestraße, Engeo, Ortsteil Bremervörde, Bremervörde, Landkreis Rotenburg, Niedersachsen, 27432, Deutschland","address":{"building":"GETI Wilba","house_number":"2","road":"Hansestraße","suburb":"Engeo","city_district":"Ortsteil Bremervörde","town":"Bremervörde","county":"Landkreis Rotenburg","state":"Niedersachsen","postcode":"27432","country":"Deutschland","country_code":"de"},"boundingbox":["53.4974704","53.4991834","9.1150452","9.1189639"]}
{"place_id":150224530,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":323206297,"lat":"53.297153002377","lon":"11.3480453571771","place_rank":26,"category":"highway"